<a href="https://colab.research.google.com/github/gellwanger/nerdcast_analysis/blob/main/Nerdcast%20Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports

In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import time
import pandas as pd
import re

## Import Selenium


In [ ]:
%%capture

!pip install selenium
!apt-get update
!apt install chromium-chromedriver
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=options)

## Get API

In [ ]:
url = 'https://jovemnerd.com.br/nerdcast/'

get_api = requests.get(url)

text = get_api.text

## Load Soup

In [ ]:
soup = bs(text,  features='html.parser')

## Load Selenium

In [ ]:
driver = webdriver.Chrome(options=options)

driver.get(url)

driver.find_element(By.CLASS_NAME, 'close').click()

time.sleep(1)

driver.find_element(By.CLASS_NAME, 'text-button-cookie').click()

time.sleep(1)

## Load Page

In [ ]:
driver.find_element(By.ID, 'show-more-podcasts').click()

time.sleep(5)

new_html = driver.page_source

soup = bs(new_html, 'html.parser')

## Loop Podcasts

In [ ]:
list_podcasts = []
list_podcasts_titles = []
list_podcasts_duration = []

i = 0

while True:
    button = driver.find_element(By.ID, 'show-more-podcasts')
    button_text = button.text

    get_podcast1 = soup.find_all('a', class_='cat-item product-nerdcast')
    temporary_list = [get_podcast1]

    if temporary_list[0][-1].text != 'NerdCast 01':
      get_podcast_temporary = soup.find_all('a', class_='cat-item product-nerdcast')
      temporary_list.append(get_podcast_temporary)

      button.click()
      time.sleep(5)
      new_html = driver.page_source
      soup = bs(new_html, 'html.parser')

      i += 1
      print(f'Iteração {i} - {temporary_list[0][-1].text}')

    else:

      time.sleep(5)
      new_html = driver.page_source
      soup = bs(new_html, 'html.parser')

      get_podcast = soup.find_all('a', class_='cat-item product-nerdcast')
      get_podcast_name = soup.find_all('h2', class_='title')
      get_podcast_duration = soup.find_all('div', class_='info')

      list_podcasts.extend(get_podcast)
      list_podcasts_titles.extend(get_podcast_name)
      list_podcasts_duration.extend(get_podcast_duration)

      print(f'Coleta concluída, com o arquivo {list_podcasts[-1].text}!')
      break

### Get Themes and Subjects

In [ ]:
list_themes = []
get_themes = soup.find('select', class_='nerdcast-filter')

for item in get_themes:
  theme = item.get_text().strip()

  if theme != '':
    list_themes.append(theme)

list_themes.pop(0)

In [ ]:
subjects = []
for item in list_themes:
  subjects.append([])

In [ ]:
list_podcasts_subject = dict(zip(list_themes, subjects))

### Loop Themes and Subjects

In [ ]:
i_theme = 1
index_theme = 0

while len(list_podcasts_subject[list_themes[-1]]) <= 0:
  select = driver.find_element(By.CLASS_NAME, 'nerdcast-filter')
  choosed = Select(select).select_by_index(i_theme)

  button = driver.find_element(By.ID, 'show-more-podcasts')

  style_value = button.get_attribute('style')

  if style_value == 'display: block;':

      button.click()
      time.sleep(5)
      new_html = driver.page_source
      soup = bs(new_html, 'html.parser')

  else:
    time.sleep(5)
    new_html = driver.page_source
    soup = bs(new_html, 'html.parser')

    get_podcast = soup.find_all('a', class_='cat-item product-nerdcast')

    list_podcasts_subject[list_themes[index_theme]] = get_podcast
    print(f'{list_themes[index_theme]}: {list_podcasts_subject[list_themes[index_theme]]}')

    i_theme += 1
    index_theme += 1

### Get text

In [ ]:
for key, value in list_podcasts_subject.items():
    names_list = []
    for item in value:
      this = item['href']
      names_list.append(this)
      list_podcasts_subject[key] = names_list

_____________________________________

# DataFrame

### df

In [ ]:
df = pd.DataFrame(columns=['Podcast', 'Link'])

In [ ]:
for item in list_podcasts:
  df.loc[df.shape[0]] = [item.text.strip(), item['href']]

In [ ]:
df = df[df['Podcast'] != '2011 - 2012 - 2022 - 2023']
df = df[df['Podcast'] != '2017 - 2020']
df = df[df['Podcast'] != '2013 - 2016']
df = df[df['Podcast'] != 'NerdCast 549 Teaser']

In [ ]:
linhas_duplicadas = df[df.duplicated(keep=False)]

In [ ]:
df = df.drop_duplicates()

## Split Podcast and Number

In [ ]:
list_of_numbers = []

for item in df['Podcast']:

  this = any(char.isdigit() for char in item)

  if this is False:
    item.index
    result_string = ''.join(item)

    list_of_numbers.append(result_string)

  else:
    pass

In [ ]:
for item in list_of_numbers:
  df = df[df['Podcast'] != item]

In [ ]:
list_text = []

for item in df['Podcast']:
  this = re.split(r'[0-9]', item)
  list_text.extend(this)

In [ ]:
list_number = []

for item in df['Podcast']:
  this = re.split(r'[a-z A-Z á-ú]', item)

  list_number.extend(this)

In [ ]:
text_new = []

for item in list_text:
  if len(item) >= 2:
    this = item.strip()
    text_new.append(this)
  else:
    pass

In [ ]:
number_new = []

number_new = [item for item in list_number if item != '']

In [ ]:
df.insert(2, column = 'Numero', value = 0)

In [ ]:
df['Numero'] = number_new
df['Podcast'] = text_new

In [ ]:
df.insert(0, 'ID', 1)
df['ID'] = range(1, len(df) + 1)

In [ ]:
df.head()

###df2

In [ ]:
df2 = pd.DataFrame(columns=['Assunto', 'Link'])

In [ ]:
for item in list_podcasts_titles:
  df2.loc[df2.shape[0]] = [item.text.strip(), item.find('a')['href']]

In [ ]:
linhas_duplicadas = df2[df2.duplicated(keep=False)]

print(linhas_duplicadas)

In [ ]:
df2 = df2.drop_duplicates()

In [ ]:
linhas_duplicadas = df2[df2.duplicated(keep=False)]

In [ ]:
df2.insert(0, 'ID', 1)
df2['ID'] = range(1, len(df2) + 1)

In [ ]:
df2.head()

###df3

In [ ]:
df3 = pd.DataFrame(columns=['Duracao', 'Data_Lancamento', 'Link'])

In [ ]:
list_duration = []
list_launch = []
list_link = []

for item in list_podcasts_duration:
  if item.find('time'):
    duracao = item.find('time').get_text().split('•')[0][1:].strip()
    list_duration.append(duracao)

    data_lancamento = item.find('time').get_text().split('•')[1][1:].strip()
    list_launch.append(data_lancamento)

    link = item.find('a')['href']
    list_link.append(link)

In [ ]:
df3['Duracao'] = list_duration
df3['Data_Lancamento'] = list_launch
df3['Link'] = list_link

In [ ]:
linhas_duplicadas = df3[df3.duplicated(keep=False)]

In [ ]:
df3 = df3.drop_duplicates()

In [ ]:
linhas_duplicadas = df[df.duplicated(keep=False)]

In [ ]:
df3.head()

### df4

In [ ]:
df4 = pd.DataFrame(columns = ['Theme', 'Link'])

In [ ]:
podcast_href_themes = []

for key, value in list_podcasts_subject.items():
  for item in value:
    podcast_href_themes.append((key, item))
print(podcast_href_themes)

In [ ]:
%%capture

for item in podcast_href_themes:
    df4 = df4.append({'Theme': item[0], 'Link': item[1]}, ignore_index=True)

In [ ]:
df4[30:60]

In [ ]:
output_file = "dados_coletados.csv"

df.to_csv(output_file, index=False)

In [ ]:
output_file = "dados_coletados2.csv"

df2.to_csv(output_file, index=False)

In [ ]:
output_file = "dados_coletados3.csv"

df3.to_csv(output_file, index=False)

In [ ]:
output_file = "dados_coletados4.csv"

df4.to_csv(output_file, index=False)